In [87]:
import pandas as pd 
import numpy as np 
import string 
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
stop = stopwords.words("english")
from nltk.stem.porter import *
from nltk.stem.porter import PorterStemmer

In [89]:
initial = pd.read_csv("initial_data.csv")

In [90]:
initial.pattern = initial.pattern.fillna("none")

In [91]:
initial.is_salary = initial.is_salary.fillna(0)

In [92]:
initial.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,sequence,postdate,description,pattern,before_balance,amount,after_balance,is_zero_delta,consumer_ref,account_ref,segment,segment_name,type,type_name,probability,is_salary,labeller
0,0,0,0,81,2019-08-27,CASHFOCUS SALARIS/SALARY,cashfocus salaris salary,236.33,6602.38,6838.71,0,111993,61809,1,Income,101,Salary,100,1,Aashray
1,1,1,1,17,2019-09-26,CASHFOCUS SALARIS/SALARY,cashfocus salaris salary,142.54,5533.09,5675.63,0,111993,61809,1,Income,101,Salary,100,1,Aashray
2,2,2,2,187,2019-04-23,AHL PAYROLL AC; SALARY,ahl payroll ac salary,27.80,11933.31,11961.11,0,50274,33344,1,Income,101,Salary,100,1,Aashray
3,3,3,3,119,2019-05-23,AHL PAYROLL AC; SALARY,ahl payroll ac salary,19.51,11522.58,11542.09,0,50274,33344,1,Income,101,Salary,100,1,Aashray
4,4,4,4,47,2019-06-21,AHL PAYROLL AC; SALARY,ahl payroll ac salary,35.39,12600.72,12636.11,0,50274,33344,1,Income,101,Salary,100,1,Aashray


## Clean patterns

In [93]:
# remove whitespace and special characters/numbers
initial['pattern'] = initial['pattern'].str.replace("\s{2,}", " ")
initial['pattern'] = initial['pattern'].str.replace("[^a-zA-Z]", " ")

# remove stopwords
initial['pattern'] = initial['pattern'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop)) 

# stem patterns
stemmer = PorterStemmer()
tokenized_pattern = initial['pattern'].apply(lambda x: x.split())
tokenized_pattern = tokenized_pattern.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
for i in range(len(tokenized_pattern)):
    tokenized_pattern[i] = ' '.join(tokenized_pattern[i])
initial['pattern'] = tokenized_pattern

In [94]:
def remove_duplicates(pattern):
    pattern = pattern.split()
    ulist = []
    [ulist.append(x) for x in pattern if x not in ulist]
    unique = ' '.join(ulist)
    return unique

In [95]:
## remove repeated words 
pattern_unique = []
for pattern in list(initial.pattern): 
    pattern_unique.append(remove_duplicates(pattern))
initial.pattern = pattern_unique

In [99]:
# remove date and number strings 
initial['pattern'] = initial['pattern'].str.replace("yymmdd| yymmdd|yymmdd ", "")
initial['pattern'] = initial['pattern'].str.replace("nnn| nnn|nnn ", "")


In [102]:
initial[initial.pattern == '']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,sequence,postdate,description,pattern,before_balance,amount,after_balance,is_zero_delta,consumer_ref,account_ref,segment,segment_name,type,type_name,probability,is_salary,labeller
169,169,169,169,378,2019-02-22,13705077,,312.49,2873.25,3185.74,0,50274,33436,7,Ignored,799,Other ignored,100,1,NaN
170,170,170,170,327,2019-03-07,50516005 0000294141,,-251.98,1900.00,1648.02,0,50274,33436,7,Ignored,799,Other ignored,100,0,NaN
171,171,171,171,311,2019-03-22,13746708,,1325.02,2873.25,4198.27,0,50274,33436,7,Ignored,799,Other ignored,100,1,NaN
172,172,172,172,211,2019-04-24,13793172,,-922.26,2873.25,1950.99,0,50274,33436,7,Ignored,799,Other ignored,100,1,NaN
173,173,173,173,128,2019-05-24,13839372,,-53.31,2873.25,2819.94,0,50274,33436,7,Ignored,799,Other ignored,100,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,23809,19451,19463,200,2019-04-08,34470284,,233.51,1000.00,1233.51,0,48293,32061,7,Ignored,799,Other ignored,100,0,Bianca
23810,23810,19452,19464,143,2019-05-09,35400636,,-1616.78,1250.00,-366.78,0,48293,32061,7,Ignored,799,Other ignored,100,0,Bianca
23811,23811,19453,19465,130,2019-05-16,50542463 0000303695,,-1889.43,2000.00,110.57,0,48293,32061,7,Ignored,799,Other ignored,100,0,Bianca
23812,23812,19454,19466,29,2019-06-27,37175882,,-1914.49,2500.00,585.51,0,48293,32061,7,Ignored,799,Other ignored,100,0,Bianca


In [105]:
initial.to_csv("temp.csv")

In [148]:
temp = pd.read_csv("temp.csv")

In [149]:
temp[temp.pattern.isnull()]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,sequence,postdate,description,pattern,before_balance,amount,...,is_zero_delta,consumer_ref,account_ref,segment,segment_name,type,type_name,probability,is_salary,labeller
169,169,169,169,169,378,2019-02-22,13705077,NaN,312.49,2873.25,...,0,50274,33436,7,Ignored,799,Other ignored,100,1,NaN
170,170,170,170,170,327,2019-03-07,50516005 0000294141,NaN,-251.98,1900.00,...,0,50274,33436,7,Ignored,799,Other ignored,100,0,NaN
171,171,171,171,171,311,2019-03-22,13746708,NaN,1325.02,2873.25,...,0,50274,33436,7,Ignored,799,Other ignored,100,1,NaN
172,172,172,172,172,211,2019-04-24,13793172,NaN,-922.26,2873.25,...,0,50274,33436,7,Ignored,799,Other ignored,100,1,NaN
173,173,173,173,173,128,2019-05-24,13839372,NaN,-53.31,2873.25,...,0,50274,33436,7,Ignored,799,Other ignored,100,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,23809,23809,19451,19463,200,2019-04-08,34470284,NaN,233.51,1000.00,...,0,48293,32061,7,Ignored,799,Other ignored,100,0,Bianca
23810,23810,23810,19452,19464,143,2019-05-09,35400636,NaN,-1616.78,1250.00,...,0,48293,32061,7,Ignored,799,Other ignored,100,0,Bianca
23811,23811,23811,19453,19465,130,2019-05-16,50542463 0000303695,NaN,-1889.43,2000.00,...,0,48293,32061,7,Ignored,799,Other ignored,100,0,Bianca
23812,23812,23812,19454,19466,29,2019-06-27,37175882,NaN,-1914.49,2500.00,...,0,48293,32061,7,Ignored,799,Other ignored,100,0,Bianca


In [150]:
temp.pattern = temp.pattern.fillna("none")

In [151]:
temp.pattern.isnull().unique()

array([False])

## Feature Creation

### Frequency of Near Identical Patterns

In [152]:
clean_patterns = list(temp.pattern)

In [153]:
clean_patterns[:10]

['cashfocu salari',
 'cashfocu salari',
 'ahl payrol ac salari',
 'ahl payrol ac salari',
 'ahl payrol ac salari',
 ' salari esko',
 ' salari esko',
 ' salari esko',
 'salari',
 'salari']

In [154]:
# generate feaure to capture frequency with which similar patterns (suggesting same employer i.e likely salaries) 
# occur for each consumer

# bag of words
vectorizer_pattern = CountVectorizer().fit_transform(clean_patterns)
vectors_pattern = vectorizer_pattern.toarray()

# similarity matrix of words 
pattern_sim = cosine_similarity(vectors_pattern) 
pattern_sim_list = list(pattern_sim)

# for each pattern, count the numbeer of patterns in the dataset tha are at least 80% similar, implying same sender
pattern_sim_num = []
for i in range(24272):
    pattern_sim_num.append(pd.DataFrame(pattern_sim_list[i]>0.8)[0].sum())
temp['pattern_sim_num'] = pattern_sim_num    



In [155]:
temp.pattern_sim_num.head(20)

0     47
1     47
2      3
3      3
4      3
5      6
6      6
7      6
8     55
9     55
10    55
11    55
12    55
13    55
14    32
15    32
16    32
17    20
18    20
19    20
Name: pattern_sim_num, dtype: int64

In [156]:
# we assume that patterns with the same similarity count are near identical
# we see that similar counts occur in clusters by consumer, where one consumer will have many matching pattterns, 
# indicative of their salary payments
# each consumer gets 0-10 salary payments, so we count the number of patterns that are 
# near identical WITHIN each consumer

similar = []
for index, row in temp.iterrows():
    search_for = row['pattern_sim_num']
    temp_pattern = temp[temp['consumer_ref'] == row['consumer_ref']]
    temp_pattern = temp_pattern[temp_pattern['pattern_sim_num'] == search_for]
    val = temp_pattern.shape[0] - 1
    similar.append(val)
temp['pattern_is_similar'] = similar

In [157]:
# boolean variable for whether matching transactions within consumer_ref are between 0-10 
# (more would be too frequent for a salary)

temp['pattern_is_similar_bool'] = ((temp['pattern_is_similar'] > 0) 
                                         &(temp['pattern_is_similar'] < 10)).astype(int)


### Frequency of Similar Transaction Amounts

In [158]:
# consistent amounts suggest regular salaries

In [159]:
# count number of rows within customer that have amounts within 10% of each other
similar = []
for index, row in temp.iterrows():
    amount_low = 0.9 * row['amount']
    amount_high = 1.1 * row['amount']
    temp_count = temp[temp['consumer_ref'] == row['consumer_ref']]
    temp_count = temp_count[temp_count['amount'] >= amount_low]
    temp_count = temp_count[temp_count['amount'] <= amount_high]
    val = temp_count.shape[0] - 1
    similar.append(val)
temp['num_similar'] = similar

# boolean for at least 2 similar amount transactions (at least 2 salaries)
temp['num_similar_bool'] = (temp['num_similar'] >2).astype(int)

### Bins for week of month in which salary is received (most salaries are received at month's end)

In [160]:
temp["postdate"] = pd.to_datetime(temp.postdate)
temp['day'] = temp['postdate'].dt.day
temp['month'] = temp['postdate'].dt.month

temp['week_bin_1'] = (temp['day'] <= 7).astype('int64')
temp['week_bin_2'] = ((temp['day'] > 7) & (temp['day'] <= 14)).astype('int64')
temp['week_bin_3'] = ((temp['day'] > 14) & (temp['day'] <= 21)).astype('int64')
temp['week_bin_4'] = ((temp['day'] > 21) & (temp['day'] <= 31)).astype('int64')

### Large transaction amount (indicative of salary)

In [161]:
temp['large_amount'] = temp.amount > 7000
temp.large_amount = temp.large_amount.astype(int)

### Regex boolean variable capturing presence of "sal"/"wage" words in pattern

In [162]:
temp['wagesal_pattern'] = temp.pattern.str.contains('^sal| wag| sal|^wag|salar|^sal$| sal |^sala| sala').astype(int)


In [163]:
temp.pattern

0                                          cashfocu salari
1                                          cashfocu salari
2                                     ahl payrol ac salari
3                                     ahl payrol ac salari
4                                     ahl payrol ac salari
                               ...                        
24267                                      interest receiv
24268    atm cash deposit marina dnr secunda watt watte...
24269                  branch cash deposit secunda mall za
24270                   atm transfer kimberley kim park za
24271    atm cash deposit dep dnr kimberl nr kim north ...
Name: pattern, Length: 24272, dtype: object

In [165]:
final_data = temp.iloc[:,4:]

In [168]:
final_data.head()

,sequence,postdate,description,pattern,before_balance,amount,after_balance,is_zero_delta,consumer_ref,account_ref,...,num_similar,num_similar_bool,day,month,week_bin_1,week_bin_2,week_bin_3,week_bin_4,large_amount,wagesal_pattern
0,81,2019-08-27,CASHFOCUS SALARIS/SALARY,cashfocu salari,236.33,6602.38,6838.71,0,111993,61809,...,0,0,27,8,0,0,0,1,0,1
1,17,2019-09-26,CASHFOCUS SALARIS/SALARY,cashfocu salari,142.54,5533.09,5675.63,0,111993,61809,...,0,0,26,9,0,0,0,1,0,1
2,187,2019-04-23,AHL PAYROLL AC; SALARY,ahl payrol ac salari,27.80,11933.31,11961.11,0,50274,33344,...,5,1,23,4,0,0,0,1,1,1
3,119,2019-05-23,AHL PAYROLL AC; SALARY,ahl payrol ac salari,19.51,11522.58,11542.09,0,50274,33344,...,5,1,23,5,0,0,0,1,1,1
4,47,2019-06-21,AHL PAYROLL AC; SALARY,ahl payrol ac salari,35.39,12600.72,12636.11,0,50274,33344,...,2,0,21,6,0,0,1,0,1,1


In [167]:
final_data.to_csv("final_data.csv")